In [ ]:

import pandas as pd

hp = pd.read_csv('/content/home_page_table.csv') 
pc = pd.read_csv('/content/payment_confirmation_table.csv') 
pp = pd.read_csv('/content/payment_page_table.csv') 
sp = pd.read_csv('/content/search_page_table.csv')
ut = pd.read_csv('/content/user_table.csv')

In [ ]:
hp = hp.rename(columns={'page': 'hp'})
pc = pc.rename(columns={'page': 'pc'})
pp = pp.rename(columns={'page': 'pp'})
sp = sp.rename(columns={'page': 'sp'})

In [ ]:
ds = ut.merge(hp, on='user_id', how='left').merge(sp, on='user_id', how='left') \
       .merge(pp, on='user_id', how='left').merge(pc, on='user_id', how='left')

In [ ]:
ds['hp'] = ds['hp'].apply(lambda x: 1 if x == 'home_page' else 0)
ds['pc'] = ds['pc'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
ds['pp'] = ds['pp'].apply(lambda x: 1 if x == 'payment_page' else 0)
ds['sp'] = ds['sp'].apply(lambda x: 1 if x == 'search_page' else 0)

In [ ]:
dsg = ds.groupby('sex')[['hp', 'sp', 'pp', 'pc']].sum().unstack('sex').unstack('sex').reset_index()
dsg.columns.name = None
dsg = dsg.rename(columns = {'index': 'page'})

In [ ]:
dsg

In [ ]:
#Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей 
#интернет-магазина по полу
from plotly import graph_objects as go
fig = go.Figure(go.Funnel(
    y = dsg['page'],
    x = dsg['Female'],
    textinfo = 'value + percent previous',
    name = 'Female'
))

fig.add_trace(go.Funnel( 
    y = dsg['page'],
    x = dsg['Male'],
    textinfo = 'value + percent previous',
    name = 'Male'
))

fig.update_layout(
    title = 'Воронка конверсии по половому признаку',
    title_x = 0.5,
    autosize = False,
    width = 1400,
    height = 700
)

fig.show()

In [ ]:
#Создать 2 новые фичи на основе колонки “date”
ds['date'] = ds['date'].apply(pd.to_datetime)
ds['month'] = ds['date'].dt.month
ds['day'] = ds['date'].dt.day_of_week()

In [ ]:
#самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).
#месяц
dpc = ds.groupby('month')[['pc']].sum().reset_index() 
dpc

In [ ]:
#самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).
#день недели
dpd = ds.groupby('day')[['pc']].sum().reset_index() 
dpd

In [ ]:
#в какой день недели лучше покупают женщины, а в какой мужчины
pd.crosstab(index = ds['day'], columns = ds['sex'], normalize = 'columns')